In [131]:
from pandas import DataFrame, concat
from pandas import read_csv
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.preprocessing import StandardScaler
import os

In [132]:
shift = 5

In [133]:
dataset = read_csv('with_timestamp.csv', header=0, index_col=0)
dataset.drop(["@timestamp", "anomaly"], axis=1, inplace=True)
dataset.head()

,system.cpu.idle.pct,system.cpu.iowait.pct,system.cpu.softirq.pct,system.cpu.system.pct,system.cpu.total.pct,system.cpu.user.pct,system.diskio_sda.iostat.await,system.diskio_sda.iostat.busy,system.diskio_sda.iostat.queue.avg_size,system.diskio_sda.iostat.read.request.merges_per_sec,...,jolokia.metrics.threading.thread_count,system.load.1,system.load.15,system.load.5,system.load.norm.1,system.load.norm.15,system.load.norm.5,system.memory.actual.used.pct,system.memory.swap.used.pct,system.memory.used.pct
0,3.5818,0.0474,0.0312,0.1058,0.4182,0.2338,0.698,12.601,0.165,0.0,...,995,0.68,0.97,1.00,0.170,0.243,0.250,0.6904,0.0,0.9620
1,3.5080,0.0934,0.0271,0.1054,0.4920,0.2661,0.572,11.104,0.137,0.0,...,995,0.58,0.96,0.97,0.145,0.240,0.243,0.6903,0.0,0.9626
2,3.5337,0.1136,0.0291,0.1025,0.4663,0.2211,0.572,11.104,0.137,0.0,...,995,0.65,0.96,0.97,0.163,0.240,0.243,0.6904,0.0,0.9628
3,3.5742,0.0503,0.0201,0.0916,0.4258,0.2637,0.572,11.104,0.137,0.0,...,995,0.68,0.95,0.95,0.170,0.238,0.238,0.6903,0.0,0.9635
4,3.5454,0.0705,0.0268,0.1051,0.4546,0.2522,0.572,11.104,0.137,0.0,...,995,0.68,0.95,0.95,0.170,0.238,0.238,0.6905,0.0,0.9638


## Set Multiple Model Parameters

In [134]:
model_epoches = 200
model_batch_size = 200

## Normalizing Data

In [135]:
dataset[dataset.columns] = StandardScaler().fit_transform(dataset)

## Shifting the dataset
shifting only one minute

In [136]:
shift_param = shift*(-6)
dataset_shifted = dataset.shift(shift_param)
dataset = dataset.iloc[:shift_param]
dataset_shifted = dataset_shifted.iloc[:shift_param]

## Spliting the dataset 
Ratio - 80 : 20

In [137]:
train = dataset.shape[0] * 2 // 10
    
train_dataset = dataset.iloc[:-train]
train_dataset_shifted = dataset_shifted.iloc[:-train]

test_dataset = dataset.iloc[-train:]
test_dataset_shifted = dataset_shifted.iloc[-train:]

In [138]:
train_dataset = train_dataset.values.reshape((train_dataset.shape[0], 1, train_dataset.shape[1]))
test_dataset_shaped = test_dataset.values.reshape((test_dataset.shape[0], 1, test_dataset.shape[1]))
print(train_dataset_shifted.shape)

(39976, 39)


## Training Models

### Feature Set 1
- system.cpu.iowait.pct'
- system.diskio_sda.iostat.await'
- system.diskio_sda.iostat.busy'
- system.diskio_sda.iostat.queue.avg_size'
- system.diskio_sda.iostat.read.request.merges_per_sec'
- system.diskio_sda.iostat.read.request.per_sec'
- system.diskio_sda.iostat.request.avg_size'
- system.diskio_sda.iostat.service_time'
- system.diskio_sda.iostat.write.request.merges_per_sec'
- system.diskio_sda.iostat.write.request.per_sec'
- system.diskio_sda2.iostat.await'
- system.diskio_sda2.iostat.busy'
- system.diskio_sda2.iostat.queue.avg_size'
- system.diskio_sda2.iostat.read.request.merges_per_sec'
- system.diskio_sda2.iostat.read.request.per_sec'
- system.diskio_sda2.iostat.request.avg_size'
- system.diskio_sda2.iostat.service_time'
- system.diskio_sda2.iostat.write.request.merges_per_sec'
- system.diskio_sda2.iostat.write.request.per_sec'

In [139]:
train_dataset_shifted_1 = train_dataset_shifted[[
    'system.cpu.iowait.pct',
    'system.diskio_sda.iostat.await',
    'system.diskio_sda.iostat.busy',
    'system.diskio_sda.iostat.queue.avg_size',
    'system.diskio_sda.iostat.read.request.merges_per_sec',
    'system.diskio_sda.iostat.read.request.per_sec',
    'system.diskio_sda.iostat.request.avg_size',
    'system.diskio_sda.iostat.service_time',
    'system.diskio_sda.iostat.write.request.merges_per_sec',
    'system.diskio_sda.iostat.write.request.per_sec',
    'system.diskio_sda2.iostat.await',
    'system.diskio_sda2.iostat.busy',
    'system.diskio_sda2.iostat.queue.avg_size',
    'system.diskio_sda2.iostat.read.request.merges_per_sec',
    'system.diskio_sda2.iostat.read.request.per_sec',
    'system.diskio_sda2.iostat.request.avg_size',
    'system.diskio_sda2.iostat.service_time',
    'system.diskio_sda2.iostat.write.request.merges_per_sec',
    'system.diskio_sda2.iostat.write.request.per_sec']]
train_dataset_shifted_1.head()

test_dataset_shifted_1 = test_dataset_shifted[[
    'system.cpu.iowait.pct',
    'system.diskio_sda.iostat.await',
    'system.diskio_sda.iostat.busy',
    'system.diskio_sda.iostat.queue.avg_size',
    'system.diskio_sda.iostat.read.request.merges_per_sec',
    'system.diskio_sda.iostat.read.request.per_sec',
    'system.diskio_sda.iostat.request.avg_size',
    'system.diskio_sda.iostat.service_time',
    'system.diskio_sda.iostat.write.request.merges_per_sec',
    'system.diskio_sda.iostat.write.request.per_sec',
    'system.diskio_sda2.iostat.await',
    'system.diskio_sda2.iostat.busy',
    'system.diskio_sda2.iostat.queue.avg_size',
    'system.diskio_sda2.iostat.read.request.merges_per_sec',
    'system.diskio_sda2.iostat.read.request.per_sec',
    'system.diskio_sda2.iostat.request.avg_size',
    'system.diskio_sda2.iostat.service_time',
    'system.diskio_sda2.iostat.write.request.merges_per_sec',
    'system.diskio_sda2.iostat.write.request.per_sec']]

In [140]:
model_lstm_1 = Sequential()
model_lstm_1.add(LSTM(10, input_shape=(1, train_dataset_shifted.shape[1])))
model_lstm_1.add(Dense(19))
model_lstm_1.compile(loss='mse', optimizer='adam')

# history = model_lstm_1.fit(train_dataset, train_dataset_shifted_1, verbose=2, shuffle=False, epochs=model_epoches, validation_data=(test_dataset_shaped ,test_dataset_shifted_1), batch_size= 200)
history = model_lstm_1.fit(train_dataset, train_dataset_shifted_1, verbose=2, shuffle=False, epochs=model_epoches)

Epoch 1/200
 - 9s - loss: 0.7430
Epoch 2/200
 - 5s - loss: 0.6886
Epoch 3/200
 - 5s - loss: 0.6439
Epoch 4/200
 - 6s - loss: 0.6063
Epoch 5/200
 - 6s - loss: 0.5750
Epoch 6/200
 - 5s - loss: 0.5486
Epoch 7/200
 - 5s - loss: 0.5279
Epoch 8/200
 - 5s - loss: 0.5088
Epoch 9/200
 - 5s - loss: 0.4942
Epoch 10/200
 - 6s - loss: 0.4825
Epoch 11/200
 - 5s - loss: 0.4732
Epoch 12/200
 - 6s - loss: 0.4686
Epoch 13/200
 - 6s - loss: 0.4607
Epoch 14/200
 - 7s - loss: 0.4562
Epoch 15/200
 - 6s - loss: 0.4527
Epoch 16/200
 - 5s - loss: 0.4497
Epoch 17/200
 - 6s - loss: 0.4463
Epoch 18/200
 - 6s - loss: 0.4449
Epoch 19/200
 - 6s - loss: 0.4429
Epoch 20/200
 - 6s - loss: 0.4428
Epoch 21/200
 - 7s - loss: 0.4411
Epoch 22/200
 - 6s - loss: 0.4413
Epoch 23/200
 - 5s - loss: 0.4375
Epoch 24/200
 - 6s - loss: 0.4353
Epoch 25/200
 - 5s - loss: 0.4397
Epoch 26/200
 - 5s - loss: 0.4338
Epoch 27/200
 - 6s - loss: 0.4325
Epoch 28/200
 - 6s - loss: 0.4282
Epoch 29/200
 - 6s - loss: 0.4373
Epoch 30/200
 - 6s - lo

### Feature Set 2
- system.cpu.total.pct'
- system.cpu.user.pct'

In [141]:
train_dataset_shifted_2 = train_dataset_shifted[[
    'system.cpu.total.pct',
    'system.cpu.user.pct']]

train_dataset_shifted_2.head()

,system.cpu.total.pct,system.cpu.user.pct
0,-0.279101,-0.217114
1,-0.246603,-0.235164
2,-0.067553,-0.196552
3,-0.198013,-0.174264
4,-0.212231,-0.215230


In [142]:
model_lstm_2 = Sequential()
model_lstm_2.add(LSTM(10, input_shape=(1, train_dataset_shifted.shape[1]), activation='softplus', recurrent_activation='linear'))
model_lstm_2.add(Dense(2))
model_lstm_2.compile(loss='mse', optimizer='adam')

model_lstm_2.fit(train_dataset, train_dataset_shifted_2, verbose=2, shuffle=False, epochs=model_epoches)

Epoch 1/200
 - 8s - loss: 1774.6043
Epoch 2/200
 - 5s - loss: 999.3749
Epoch 3/200
 - 5s - loss: 155.4733
Epoch 4/200
 - 5s - loss: 58.3108
Epoch 5/200
 - 5s - loss: 107.1113
Epoch 6/200
 - 5s - loss: 1.9421
Epoch 7/200
 - 5s - loss: 13.7248
Epoch 8/200
 - 5s - loss: 58.8871
Epoch 9/200
 - 5s - loss: 6.3000
Epoch 10/200
 - 5s - loss: 0.8512
Epoch 11/200
 - 5s - loss: 0.7724
Epoch 12/200
 - 5s - loss: 0.5874
Epoch 13/200
 - 5s - loss: 0.3688
Epoch 14/200
 - 5s - loss: 0.2463
Epoch 15/200
 - 5s - loss: 0.3263
Epoch 16/200
 - 5s - loss: 0.2472
Epoch 17/200
 - 5s - loss: 0.2206
Epoch 18/200
 - 5s - loss: 0.2256
Epoch 19/200
 - 5s - loss: 0.2027
Epoch 20/200
 - 5s - loss: 0.1792
Epoch 21/200
 - 5s - loss: 0.1819
Epoch 22/200
 - 5s - loss: 0.1359
Epoch 23/200
 - 5s - loss: 0.1423
Epoch 24/200
 - 5s - loss: 0.1510
Epoch 25/200
 - 5s - loss: 0.1122
Epoch 26/200
 - 5s - loss: 0.1134
Epoch 27/200
 - 5s - loss: 0.0990
Epoch 28/200
 - 5s - loss: 0.1003
Epoch 29/200
 - 5s - loss: 0.0928
Epoch 30/20

### Feature Set 3
- jolokia.metrics.memory.heap_memory_usage.committed'
- jolokia.metrics.memory.heap_memory_usage.max'

In [143]:
train_dataset_shifted_3 = train_dataset_shifted[[
    'jolokia.metrics.memory.heap_memory_usage.committed',
    'jolokia.metrics.memory.heap_memory_usage.max']]

train_dataset_shifted_3.head()

,jolokia.metrics.memory.heap_memory_usage.committed,jolokia.metrics.memory.heap_memory_usage.max
0,0.808558,0.808558
1,0.206531,0.206531
2,0.206531,0.206531
3,0.206531,0.206531
4,0.206531,0.206531


In [144]:
model_lstm_3 = Sequential()
model_lstm_3.add(LSTM(10, input_shape=(1, train_dataset_shifted.shape[1]), activation='softplus', recurrent_activation='linear'))
model_lstm_3.add(Dense(2))
model_lstm_3.compile(loss='mse', optimizer='adam')

model_lstm_3.fit(train_dataset, train_dataset_shifted_3, verbose=2, shuffle=False, epochs=model_epoches)

Epoch 1/200
 - 8s - loss: 197.5604
Epoch 2/200
 - 5s - loss: 12.7361
Epoch 3/200
 - 5s - loss: 3.7213
Epoch 4/200
 - 5s - loss: 2.2442
Epoch 5/200
 - 5s - loss: 154.7669
Epoch 6/200
 - 5s - loss: 1.0877
Epoch 7/200
 - 5s - loss: 0.8650
Epoch 8/200
 - 5s - loss: 0.8230
Epoch 9/200
 - 5s - loss: 0.7936
Epoch 10/200
 - 5s - loss: 0.8041
Epoch 11/200
 - 5s - loss: 0.8241
Epoch 12/200
 - 5s - loss: 0.8028
Epoch 13/200
 - 5s - loss: 0.7833
Epoch 14/200
 - 5s - loss: 0.7643
Epoch 15/200
 - 5s - loss: 0.8505
Epoch 16/200
 - 5s - loss: 0.7915
Epoch 17/200
 - 5s - loss: 0.7045
Epoch 18/200
 - 5s - loss: 0.7092
Epoch 19/200
 - 5s - loss: 0.7196
Epoch 20/200
 - 5s - loss: 0.7039
Epoch 21/200
 - 5s - loss: 0.6966
Epoch 22/200
 - 5s - loss: 1.9564
Epoch 23/200
 - 5s - loss: 0.6911
Epoch 24/200
 - 5s - loss: 0.6945
Epoch 25/200
 - 5s - loss: 0.6818
Epoch 26/200
 - 5s - loss: 0.6894
Epoch 27/200
 - 5s - loss: 0.6854
Epoch 28/200
 - 5s - loss: 0.6891
Epoch 29/200
 - 5s - loss: 0.6737
Epoch 30/200
 - 5s

### Feature Set 4
- system.load.1'
- system.load.15'
- system.load.5'
- system.load.norm.1'
- system.load.norm.15'
- system.load.norm.5'

In [145]:
train_dataset_shifted_4 = train_dataset_shifted[[
    'system.load.1',
    'system.load.15',
    'system.load.5',
    'system.load.norm.1',
    'system.load.norm.15',
    'system.load.norm.5']]

train_dataset_shifted_4.head()

,system.load.1,system.load.15,system.load.5,system.load.norm.1,system.load.norm.15,system.load.norm.5
0,-0.152295,-0.133203,-0.134026,-0.154622,-0.136080,-0.136533
1,-0.156706,-0.134027,-0.135839,-0.159032,-0.136740,-0.138466
2,-0.160567,-0.134851,-0.137652,-0.163002,-0.137729,-0.140158
3,-0.044208,-0.123311,-0.111068,-0.046567,-0.126188,-0.113576
4,-0.044208,-0.123311,-0.111068,-0.046567,-0.126188,-0.113576


In [146]:
model_lstm_4 = Sequential()
model_lstm_4.add(LSTM(10, input_shape=(1, train_dataset_shifted.shape[1]), activation='softplus', recurrent_activation='linear'))
model_lstm_4.add(Dense(6))
model_lstm_4.compile(loss='mse', optimizer='adam')

model_lstm_4.fit(train_dataset, train_dataset_shifted_4, verbose=2, shuffle=False, epochs=model_epoches)

Epoch 1/200
 - 8s - loss: 162.5273
Epoch 2/200
 - 5s - loss: 39.7505
Epoch 3/200
 - 5s - loss: 2.3658
Epoch 4/200
 - 5s - loss: 1.8410
Epoch 5/200
 - 5s - loss: 1.2919
Epoch 6/200
 - 5s - loss: 0.8720
Epoch 7/200
 - 5s - loss: 1.6080
Epoch 8/200
 - 5s - loss: 0.4489
Epoch 9/200
 - 5s - loss: 0.2248
Epoch 10/200
 - 5s - loss: 0.1920
Epoch 11/200
 - 5s - loss: 0.1851
Epoch 12/200
 - 5s - loss: 0.2594
Epoch 13/200
 - 5s - loss: 0.4523
Epoch 14/200
 - 5s - loss: 0.3518
Epoch 15/200
 - 5s - loss: 0.1922
Epoch 16/200
 - 5s - loss: 0.2025
Epoch 17/200
 - 5s - loss: 0.2016
Epoch 18/200
 - 5s - loss: 0.2152
Epoch 19/200
 - 5s - loss: 0.2641
Epoch 20/200
 - 5s - loss: 0.1849
Epoch 21/200
 - 5s - loss: 0.1614
Epoch 22/200
 - 5s - loss: 0.1816
Epoch 23/200
 - 5s - loss: 0.1570
Epoch 24/200
 - 5s - loss: 0.1625
Epoch 25/200
 - 5s - loss: 0.1685
Epoch 26/200
 - 5s - loss: 0.1771
Epoch 27/200
 - 5s - loss: 0.1537
Epoch 28/200
 - 5s - loss: 0.7213
Epoch 29/200
 - 5s - loss: 0.1346
Epoch 30/200
 - 5s -

### Feature Set 5
- system.cpu.idle.pct'
- system.cpu.softirq.pct'
- system.cpu.system.pct'
- jolokia.metrics.memory.heap_memory_usage.used'
- jolokia.metrics.memory.non_heap_memory_usage.used'
- jolokia.metrics.threading.daemon_thread_count'
- jolokia.metrics.threading.thread_count'
- system.memory.actual.used.pct'
- system.memory.swap.used.pct'
- system.memory.used.pct'

In [147]:
train_dataset_shifted_5 = train_dataset_shifted[[
    'system.cpu.idle.pct',
    'system.cpu.softirq.pct',
    'system.cpu.system.pct',
    'jolokia.metrics.memory.heap_memory_usage.used',
    'jolokia.metrics.memory.non_heap_memory_usage.used',
    'jolokia.metrics.threading.daemon_thread_count',
    'jolokia.metrics.threading.thread_count',
    'system.memory.actual.used.pct',
    'system.memory.swap.used.pct',
    'system.memory.used.pct']]

train_dataset_shifted_5.head()

,system.cpu.idle.pct,system.cpu.softirq.pct,system.cpu.system.pct,jolokia.metrics.memory.heap_memory_usage.used,jolokia.metrics.memory.non_heap_memory_usage.used,jolokia.metrics.threading.daemon_thread_count,jolokia.metrics.threading.thread_count,system.memory.actual.used.pct,system.memory.swap.used.pct,system.memory.used.pct
0,0.279101,-0.378447,-0.133873,-1.005035,-1.670330,-0.459191,-1.10143,-1.037900,-0.078425,0.334706
1,0.246603,0.141898,0.795873,-0.204441,-1.669447,-0.459191,-1.10143,-1.033430,-0.078425,0.386437
2,0.067553,0.364903,-0.392135,-0.204441,-1.669447,-0.459191,-1.10143,-1.042371,-0.078425,0.386437
3,0.198013,-0.155442,0.709786,-0.204441,-1.669447,-0.459191,-1.10143,-1.037900,-0.078425,0.438169
4,0.212231,-0.254556,-0.047785,-0.204441,-1.669447,-0.459191,-1.10143,-1.028959,-0.078425,0.467729


In [148]:
model_lstm_5 = Sequential()
model_lstm_5.add(LSTM(10, input_shape=(1, train_dataset_shifted.shape[1]), activation='softplus', recurrent_activation='linear'))
model_lstm_5.add(Dense(10))
model_lstm_5.compile(loss='mse', optimizer='adam')

model_lstm_5.fit(train_dataset, train_dataset_shifted_5, verbose=2, shuffle=False, epochs=model_epoches)

Epoch 1/200
 - 9s - loss: 539.2354
Epoch 2/200
 - 5s - loss: 10.2317
Epoch 3/200
 - 5s - loss: 4.8082
Epoch 4/200
 - 5s - loss: 5.9810
Epoch 5/200
 - 5s - loss: 4.9344
Epoch 6/200
 - 5s - loss: 2.3154
Epoch 7/200
 - 5s - loss: 1.3939
Epoch 8/200
 - 5s - loss: 0.8941
Epoch 9/200
 - 5s - loss: 0.7023
Epoch 10/200
 - 5s - loss: 0.5849
Epoch 11/200
 - 5s - loss: 0.5426
Epoch 12/200
 - 5s - loss: 0.5346
Epoch 13/200
 - 5s - loss: 0.5524
Epoch 14/200
 - 5s - loss: 0.5264
Epoch 15/200
 - 5s - loss: 0.5081
Epoch 16/200
 - 5s - loss: 0.4877
Epoch 17/200
 - 5s - loss: 0.4808
Epoch 18/200
 - 5s - loss: 0.4801
Epoch 19/200
 - 5s - loss: 0.4763
Epoch 20/200
 - 5s - loss: 0.4652
Epoch 21/200
 - 5s - loss: 0.4632
Epoch 22/200
 - 5s - loss: 0.4666
Epoch 23/200
 - 5s - loss: 0.4560
Epoch 24/200
 - 5s - loss: 0.4580
Epoch 25/200
 - 5s - loss: 0.4502
Epoch 26/200
 - 5s - loss: 0.4535
Epoch 27/200
 - 5s - loss: 0.4494
Epoch 28/200
 - 5s - loss: 0.4561
Epoch 29/200
 - 5s - loss: 0.4444
Epoch 30/200
 - 5s -

## Serializing Models

In [149]:
directory = "shift-"+str(shift)
models = [model_lstm_1, model_lstm_2, model_lstm_3, model_lstm_4, model_lstm_5]

if not os.path.exists(directory):
    os.makedirs(directory)

count = 0
for model_to_save in models:
    count += 1
    model_json = model_to_save.to_json()
    with open(directory+"/model_lstm_"+str(count)+".json", "w") as json_file:
        json_file.write(model_json)

        # serialize weights to HDF5
    model_to_save.save_weights(directory+"/model_lstm_"+str(count)+".h5")

print("Saved models to disk")

Saved models to disk
